# Семинар 2. Модель данных Python: всё — объект

## 🎯 Цели семинара
- Закрепить понимание мутабельности и иммутабельности объектов
- Научиться работать с пространствами имён и областями видимости
- Освоить инструменты интроспекции: `type()`, `id()`, `dir()`, `isinstance()`
- Научиться правильно копировать объекты

---

## 📝 Разминка (3 обычные задачи)

### Задача 1: Детектор мутабельности
**Уровень:** Начальный

Напишите функцию `is_mutable(obj)`, которая:
- Принимает любой объект Python
- Возвращает `True`, если объект мутабельный, и `False`, если иммутабельный
- Не используйте жестко заданные списки типов (не проверяйте `type(obj) in [list, dict, ...]`)
- Подсказка: попробуйте изменить объект и посмотреть, изменится ли его `id()`

**Пример:**
```python
>>> is_mutable([1, 2, 3])
True
>>> is_mutable("hello")
False
>>> is_mutable((1, 2, 3))
False
>>> is_mutable({"a": 1})
True
```

---

### Задача 2: Анализатор памяти  
**Уровень:** Начальный

Напишите функцию `memory_analyzer(*objects)`, которая:
- Принимает произвольное количество объектов
- Возвращает словарь с информацией:
  - `total_objects`: общее количество уникальных объектов
  - `shared_count`: количество объектов, на которые ссылается более одного аргумента
  - `memory_map`: словарь `{id(obj): [ссылающиеся_переменные]}`
- Имена переменных можно заменить на `arg1`, `arg2` и т.д.

**Пример:**
```python
a = [1, 2, 3]
b = a
c = "hello"

>>> memory_analyzer(a, b, c)
{
    'total_objects': 2,
    'shared_count': 1,
    'memory_map': {
        1402456789012: ['arg1', 'arg2'],  # список [1, 2, 3]
        1402456789123: ['arg3']          # строка "hello"
    }
}
```

---

### Задача 3: Пространство имён декоратора
**Уровень:** Начальный

Создайте декоратор `namespace_logger`, который:
1. При вызове декорированной функции выводит:
   - Имя функции
   - Локальные переменные до выполнения функции
   - Локальные переменные после выполнения функции
   - Глобальные переменные, к которым обращалась функция
2. Не изменяет поведение оригинальной функции
3. Использует `locals()` и `globals()` для получения информации

**Пример:**
```python
@namespace_logger
def calculate(a, b):
    result = a + b
    message = f"{a} + {b} = {result}"
    return message

>>> calculate(5, 3)
Имя функции: calculate
Локальные переменные до: {'a': 5, 'b': 3}
Локальные переменные после: {'a': 5, 'b': 3, 'result': 8, 'message': '5 + 3 = 8'}
Глобальные переменные: {'__name__': '__main__', ...}
'5 + 3 = 8'
```

---

## 🧠 Для подумать (2 задачи повышенной сложности)

### Задача 4: Утиный детектор
**Уровень:** Средний

Реализуйте функцию `duck_checker(obj, interface)`, которая:
- Проверяет, соответствует ли объект заданному интерфейсу (утиная типизация)
- `interface` — словарь `{имя_метода: количество_аргументов}`
- Возвращает `True`, если объект имеет все методы с правильной сигнатурой
- Должна работать с любыми callable объектами (функции, методы, классы, лямбды)
- Учитывайте, что методы могут иметь значения по умолчанию!

**Пример:**
```python
# Проверяем, похож ли объект на список
>>> duck_checker([], {'append': 1, 'pop': 0, 'extend': 1})
True

>>> duck_checker("строка", {'upper': 0, 'split': 1})
True

# Создаем класс с методом, у которого есть значения по умолчанию
class Test:
    def method(self, x, y=10):
        return x + y

>>> duck_checker(Test(), {'method': 1})  # method можно вызвать с 1 аргументом
True
```

---

### Задача 5: Глубокий компаратор
**Уровень:** Средний

Напишите функцию `deep_equals(a, b, visited=None)`, которая:
1. Рекурсивно сравнивает два объекта на глубокое равенство
2. Должна корректно работать с циклическими ссылками
3. Учитывает как значения, так и структуру
4. Поддерживает основные типы: списки, словари, кортежи, множества
5. Использует параметр `visited` для отслеживания уже посещённых пар объектов

**Пример:**
```python
# Простой случай
>>> deep_equals([1, 2, 3], [1, 2, 3])
True

# С циклическими ссылками
a = []
a.append(a)  # a ссылается сам на себя

b = []
b.append(b)

>>> deep_equals(a, b)
True

# Сложные структуры
>>> deep_equals(
    {'a': [1, 2, {'x': 10}], 'b': (1, 2)},
    {'a': [1, 2, {'x': 10}], 'b': (1, 2)}
)
True
```

---

## 🏠 Домашнее задание (2 задачи)

### Задача 6: Иммутабельный буфер
Создайте класс `ImmutableBuffer`, который имитирует поведение иммутабельного объекта:
1. При создании принимает произвольные данные
2. При попытке изменить атрибуты выбрасывает исключение `AttributeError`
3. Позволяет читать данные через индексацию и итерацию
4. Поддерживает метод `copy()` для создания копии
5. Поддерживает метод `mutate(**changes)` для создания нового объекта с изменениями (как строки)

**Требования:**
```python
>>> buf = ImmutableBuffer([1, 2, 3, 4, 5])
>>> buf[0]
1
>>> buf[1:3]
[2, 3]

# Попытка изменить
>>> buf[0] = 10
AttributeError: ImmutableBuffer object does not support item assignment

# Создание изменённой копии
>>> new_buf = buf.mutate(data=[10, 2, 3, 4, 5])
>>> new_buf[0]
10
>>> buf[0]  # оригинал не изменился
1

# Поддержка итерации
>>> for item in buf:
...     print(item)
1
2
3
4
5
```

### Задача 7: Анализатор зависимостей
Создайте функцию `analyze_dependencies(func)`, которая:
1. Принимает функцию в качестве аргумента
2. Возвращает словарь с информацией о зависимостях функции:
   - `globals_used`: список глобальных переменных, которые использует функция
   - `builtins_used`: список встроенных функций, которые использует функция
   - `closures`: словарь с переменными из замыканий (если есть)
   - `defaults`: значения по умолчанию для параметров
   - `annotations`: аннотации типов параметров
3. Используйте инструменты интроспекции для анализа объекта функции

**Пример:**
```python
x = 10
y = 20

def example(a: int, b: int = 5) -> int:
    """Пример функции с зависимостями."""
    result = a + b + x + len(str(y))
    return result

>>> analyze_dependencies(example)
{
    'globals_used': ['x', 'y'],
    'builtins_used': ['len', 'str'],
    'closures': {},
    'defaults': (5,),
    'annotations': {'return': int, 'a': int, 'b': int},
    'docstring': 'Пример функции с зависимостями.'
}

# Для функции с замыканием
def outer(z):
    def inner():
        return z + 1
    return inner

>>> analyze_dependencies(outer(5))
{
    'globals_used': [],
    'builtins_used': [],
    'closures': {'z': 5},
    'defaults': None,
    'annotations': {},
    'docstring': None
}
```

---

## 📋 Критерии оценки

### Обычные задачи (1-3):
- Корректность решения: 60%
- Эффективность кода: 20%
- Читаемость и соответствие PEP 8: 20%

### Задачи для подумать (4-5):
- Корректность: 40%
- Обработка крайних случаев: 30%
- Оптимизация: 20%
- Читаемость кода: 10%

### Домашнее задание (6-7):
- Полная реализация требований: 50%
- Корректность работы: 30%
- Обработка ошибок: 10%
- Соответствие PEP 8: 10%

---

## 💡 Советы по выполнению

### Для задач 1-3:
1. **Задача 1:** Вспомните, что иммутабельные объекты нельзя изменить "на месте". Попробуйте добавить элемент в список и в кортеж и посмотрите, что происходит с `id()`.
2. **Задача 2:** Используйте `id()` для определения уникальности объектов. Создайте словарь, где ключом будет идентификатор объекта.
3. **Задача 3:** Изучите, как работают `locals()` и `globals()`. Помните, что декоратор должен возвращать функцию-обёртку.

### Для задач 4-5:
4. **Задача 4:** Используйте `hasattr()` и `getattr()`. Для проверки количества аргументов используйте `inspect.signature()`.
5. **Задача 5:** Используйте рекурсию. Для отслеживания циклических ссылок сохраняйте пары `(id(a), id(b))` в множестве.

### Для домашнего задания:
6. **Задача 6:** Переопределите магические методы `__setitem__`, `__delitem__`, `__setattr__` для запрета изменений.
7. **Задача 7:** Изучите атрибуты функций: `__code__`, `__globals__`, `__closure__`, `__defaults__`, `__annotations__`.

---

## 🚀 Дополнительные вызовы (по желанию)

### Для отличников:
1. **К задаче 4:** Добавьте поддержку проверки типов аргументов (используя аннотации).
2. **К задаче 5:** Добавьте поддержку пользовательских классов и сравнения по значению (`__eq__`).
3. **К задаче 6:** Реализуйте кэширование хэшей для объектов `ImmutableBuffer`.
4. **К задаче 7:** Добавьте анализ сложности функции (количество операций, вложенность вызовов).

---

## 📅 Сроки выполнения

- **Задачи 1-5:** К следующему семинару
- **Домашнее задание 6-7:** 2 недели

**Формат сдачи:**
1. Создайте папку `seminar_02_solutions`
2. Для задач 1-5 создайте файл `tasks.py`
3. Для домашнего задания создайте файлы `immutable_buffer.py` и `dependency_analyzer.py`
4. Добавьте файл `README.md` с примерами использования
5. Загрузите в свой репозиторий на GitHub
6. Пришлите ссылку в LMS

---

<div style="background: #f8f9fa; padding: 15px; border-left: 4px solid #4CAF50; border-radius: 5px; margin-top: 20px;">
<strong>📞 Вопросы и помощь:</strong><br>
1. Создайте issue в репозитории курса с тегом "seminar_02"<br>
2. Задавайте вопросы на практических занятиях<br>
3. Обсуждайте в Telegram-чате группы<br><br>

<strong>🚨 Важно:</strong> Код должен быть вашим собственным. Можно обсуждать подходы и идеи, но не копировать решения одногруппников.
</div>